In [1]:
pip install elasticsearch


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': 'ccd21680e57d', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'y62WqWXRRWKajl8WN06--w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [3]:
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [14]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [5]:
query = "How do execute a command on a Kubernetes pod?"
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)
print(response['hits']['hits'][0]['_score'])

44.50556


In [6]:
query = "How do copy a file to a Docker container?"
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
            "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)
result_docs = []
    
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [7]:
context_template = """
Q: {question}
A: {text}
""".strip()

content = "\n\n".join(
    context_template.format(**entry) for entry in result_docs
)
print(content)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [8]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query,context=content)
print(len(prompt))

1446


In [9]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/contain

In [32]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [36]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt)
len(tokens)

320

In [37]:
pip install ollama


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import ollama
def llm(prompt):
    response = ollama.chat(
        model='llama2',  # or any model you've pulled, like 'mistral', 'phi3', etc.
        messages=[{"role": "user", "content": prompt}]
    )
    print(response)
    return response['message']['content']

llm(prompt)

model='llama2' created_at='2025-06-19T10:40:38.201849Z' done=True done_reason='stop' total_duration=41553244500 load_duration=1099791500 prompt_eval_count=408 prompt_eval_duration=15947023333 eval_count=249 eval_duration=24493304500 message=Message(role='assistant', content='To answer the question "How do copy a file to a Docker container?", we can look at the context provided in the FAQ database.\n\nFrom the context, we know that there are two ways to copy files into a Docker container:\n\n1. Using the `docker cp` command: This command allows you to copy a file or directory from your local machine into a running Docker container. The basic syntax is `docker cp /path/to/local/file_or_directory container_id:/path/in/container`.\n2. Using the `COPY` command in the Dockerfile: This command allows you to copy files from your local machine into the container\'s working directory during the build process. The basic syntax is `COPY ["src/predict.py", "models/xgb_model.bin", "./"]`.\n\nBoth of

'To answer the question "How do copy a file to a Docker container?", we can look at the context provided in the FAQ database.\n\nFrom the context, we know that there are two ways to copy files into a Docker container:\n\n1. Using the `docker cp` command: This command allows you to copy a file or directory from your local machine into a running Docker container. The basic syntax is `docker cp /path/to/local/file_or_directory container_id:/path/in/container`.\n2. Using the `COPY` command in the Dockerfile: This command allows you to copy files from your local machine into the container\'s working directory during the build process. The basic syntax is `COPY ["src/predict.py", "models/xgb_model.bin", "./"]`.\n\nBoth of these methods can be used to copy a file into a Docker container. Therefore, the answer to the question "How do copy a file to a Docker container?" is:\n\nYou can use either the `docker cp` command or the `COPY` command in the Dockerfile to copy a file into a Docker contain